In [2]:
import pandas as pd
import numpy as np
import json
!pip install pytextrank scattertext

import spacy
from scattertext import SampleCorpora, PhraseMachinePhrases, dense_rank, RankDifference, AssociationCompactor, produce_scattertext_explorer
from scattertext.CorpusFromPandas import CorpusFromPandas
import pytextrank, spacy
import scattertext as st
nlp = spacy.load('en')

import os
os.chdir('/content/drive/My Drive/FearSpeech')


     |████████████████████████████████| 6.9MB 16.7MB/s 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df=pd.read_csv("Final_annotated_eng.csv")
#del(dataFrame["Unnamed: 0"])

In [0]:
label=[]
text=[]
for i in range(0, len(df)):
    text.append(df['text'][i])
    label.append(df['label'][i])


In [0]:
dataFrame = pd.DataFrame(list(zip(text, label)), 
               columns =['text', 'label']) 

In [22]:
dataFrame.head()

,text,label
0,# If you keep drums Tasha beaten Tajiya in Muh...,1
1,Watch the video carefully. Muslims are taking...,1
2,👉 ▶ 🔵 🔵 👉 🔺 🔺 🔺 🔺 🔺 🔺 🔵 🔵 🔵 🔵 ...,0
3,"* Forgotten memories, forgotten by everyone. *...",0
4,* Administrator 🚩 ✊ 📚 📚 ✒ 📚 📚 📚 📚 ✒ 📚 ...,0


In [0]:
convention_df = dataFrame.assign(
    parse=lambda df: df.text.apply(nlp),
    party=lambda df: df.label.apply({1.0: 'FearSpeech', 0.0: 'NormalSpeech'}.get)
)

In [24]:
convention_df.head()

,text,label,parse,party
0,# If you keep drums Tasha beaten Tajiya in Muh...,1,"(#, If, you, keep, drums, Tasha, beaten, Tajiy...",FearSpeech
1,Watch the video carefully. Muslims are taking...,1,"(Watch, the, video, carefully, ., , Muslims, ...",FearSpeech
2,👉 ▶ 🔵 🔵 👉 🔺 🔺 🔺 🔺 🔺 🔺 🔵 🔵 🔵 🔵 ...,0,"( , 👉, , ▶, , 🔵, , 🔵, , 👉, , 🔺, , 🔺, , ...",NormalSpeech
3,"* Forgotten memories, forgotten by everyone. *...",0,"(*, Forgotten, memories, ,, forgotten, by, eve...",NormalSpeech
4,* Administrator 🚩 ✊ 📚 📚 ✒ 📚 📚 📚 📚 ✒ 📚 ...,0,"(*, Administrator, 🚩, , ✊, 📚, 📚, ✒, 📚, 📚, , ...",NormalSpeech


In [25]:
corpus = st.CorpusFromParsedDocuments(
    convention_df,
    category_col='party',
    parsed_col='parse',
    feats_from_spacy_doc=st.PyTextRankPhrases()
).build(
).compact(
    AssociationCompactor(2000, use_non_text_features=True)
)

/usr/local/lib/python3.6/dist-packages/scattertext/termscoring/ScaledFScore.py:129: RuntimeWarning: invalid value encountered in true_divide
  precision = (cat_word_counts * 1. / (cat_word_counts + not_cat_word_counts))
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


In [26]:
term_category_scores = corpus.get_metadata_freq_df('')
print(term_category_scores)

                        FearSpeech  NormalSpeech
term                                            
😬                         1.398395      0.000000
😐 #                       0.106158      0.000000
#                         0.187711      0.031973
wrong apply 😬  animals    0.087047      0.000000
wrong anchor              0.083790      0.000000
...                            ...           ...
barra                     0.018490      0.000000
muhammed                  0.036980      0.000000
the sky                   0.016999      0.000000
the names                 0.016416      0.000000
only 13 wives             0.012991      0.000000

[2000 rows x 2 columns]


In [0]:
term_ranks = np.argsort(np.argsort(-term_category_scores, axis=0), axis=0) + 1
metadata_descriptions = {
    term: '<br/>' + '<br/>'.join(
        '<b>%s</b> TextRank score rank: %s/%s' % (cat, term_ranks.loc[term, cat], corpus.get_num_metadata())
        for cat in corpus.get_categories())
    for term in corpus.get_metadata()
}

In [0]:
category_specific_prominence = term_category_scores.apply(
    lambda r: r.FearSpeech if r.FearSpeech > r.NormalSpeech else -r.NormalSpeech,
    axis=1
)

In [0]:
html1 = produce_scattertext_explorer(
    corpus,
    category='FearSpeech',
    minimum_term_frequency=0,
    pmi_threshold_coefficient=0,
    width_in_pixels=1000,
    transform=dense_rank,
    #metadata=corpus.get_df()['speaker'],
    scores=category_specific_prominence,
    sort_by_dist=False,
    use_non_text_features=True,
    topic_model_term_lists={term: [term] for term in         
                            corpus.get_metadata()},
    topic_model_preview_size=0,
    metadata_descriptions=metadata_descriptions,
    use_full_doc=True
)

In [30]:
open("Convention-Visualization1.html", 'wb').write(html1.encode('utf-8'))

1427427

In [0]:
html2 = produce_scattertext_explorer(
    corpus,
    category='FearSpeech',
    minimum_term_frequency=0,
    pmi_threshold_coefficient=0,
    width_in_pixels=1000,
    transform=dense_rank,
    use_non_text_features=True,
    #metadata=corpus.get_df()['speaker'],
    term_scorer=RankDifference(),
    sort_by_dist=False,
    topic_model_term_lists={term: [term] for term in 
                            corpus.get_metadata()},
    topic_model_preview_size=0, 
    metadata_descriptions=metadata_descriptions,
    use_full_doc=True
)

In [32]:
open("Convention-Visualization2.html", 'wb').write(html2.encode('utf-8'))

1425920

In [0]:
import spacy
from scattertext import SampleCorpora, PhraseMachinePhrases, dense_rank, RankDifference, AssociationCompactor, produce_scattertext_explorer
from scattertext.CorpusFromPandas import CorpusFromPandas


In [0]:
corpus = (CorpusFromPandas(convention_df,
                           category_col='party',
                           text_col='text',
                           feats_from_spacy_doc=PhraseMachinePhrases(),
                           nlp=spacy.load('en', parser=False))
          .build().compact(AssociationCompactor(4000)))


In [35]:
html3 = produce_scattertext_explorer(corpus,
                                    category='FearSpeech',
                                    category_name='FearSpeech',
                                    not_category_name='NormalSpeech',
                                    minimum_term_frequency=0,
                                    pmi_threshold_coefficient=0,
                                    transform=dense_rank,
                                    #metadata=corpus.get_df()['speaker'],
                                    term_scorer=RankDifference(),
                                    width_in_pixels=1000)

/usr/local/lib/python3.6/dist-packages/scattertext/termscoring/ScaledFScore.py:296: RuntimeWarning: invalid value encountered in true_divide
  recall = cat_word_counts * 1. / cat_word_counts.sum()


In [36]:
open("Convention-Visualization3.html", 'wb').write(html3.encode('utf-8'))

923405